In [1]:
import random
import math
import binascii
import numpy as np
from PIL import Image

In [2]:
def import_file(filename):    
    extension = filename.split('.')[1]
    Bytes = np.fromfile(filename,dtype="uint8")
    Bits = np.unpackbits(Bytes)
    bits=[]
    for i in range(0,len(Bits),1):
        bits.append(Bits[i])
    return bits,extension

In [3]:
file=input("Enter File to Decrypt:")
if(file.split('.')[1]=="jpg" or file.split('.')[1]=="png"):
    im=Image.open(file,"r").convert("1") 
    extension=file.split('.')[1]
    value = list(im.getdata())
    for i in range(0,len(value),1):
        if(value[i]==255):
            value[i]=1
else:
    value,extension=import_file(file)

Enter File to Decrypt:enc.jpg


In [4]:
keybits=input("Enter Key String:")
key=binascii.unhexlify(keybits)
#print(key,type(key))
key=key.decode('utf-8')
key1 =key.split('$')
XL=int(key1[0])
YL=int(key1[1])
r=int(key1[2])
P=int(key1[3])
rotate=[]
for i in range(4,len(key1),1):
    if(key1[i]==''):
        continue
    rotate.append(int(key1[i]))
rotate =rotate[::-1]

Enter Key String:343030243430302437302431333934373224323730243237302431383024313830243237302439302439302439302439302430243336302439302433363024


In [5]:
xy=[]
for i in range(0,XL+1,1):
    for j in range(0,YL+1,1):
        xy.append(str(i)+","+str(j))        

In [6]:
for i in range(-(r),XL+1+r,1):
    for j in range(-(r),YL+1+r,1):
        if(i<0 or j<0 or i>XL or j>YL):
            xy.append(str(i)+","+str(j))

In [7]:
#print(xy)

In [8]:
def sort1(val):
    return val[1]
circles=[]
for i in range(0,XL+1,r+r):
    for j in range(0,YL+1,r+r):
        circles.append([i,j])
for i in range(r,XL+1-r,r+r):
    for j in range(r,YL+1-r,r+r):
        circles.append([i,j])
circles.sort(key=sort1)

In [9]:
#print(circles)

In [10]:
points=[]
for a in range(0,len(circles),1):
    h = circles[a][0]
    k = circles[a][1]
    d = 3-(2*r) 
    x = 0
    y = r
    while(y>=x):
        if(d > 0):
            x += 1
            y -= 1
            d += 4*(x-y)+10
        if(d<0):
            x += 1
            d += (4*x)+6
        points.append([h+x,k+y])
        points.append([h-x,k+y])
        points.append([h+x,k-y])
        points.append([h-x,k-y])
        points.append([h+y,k+x])
        points.append([h-y,k+x])
        points.append([h+y,k-x])
        points.append([h-y,k-x])

    points.append("e")

In [11]:
circles=circles[::-1]
#print(circles)
points.pop()
points=points[::-1]
points.append('e')
#print(points)

In [12]:
c=0;
for a in range(0,len(circles),1):
    h=circles[a][0]
    k=circles[a][1]
    rotpoints=[]
    for j in range(c,points.index('e')+1,1):
        if(points[j]=='e'):
            break
        else:
            rotpoints.append([points[j][0],points[j][1], value[xy.index(str(points[j][0])+","+str(points[j][1]))]])
    counter=0
    for i in range(c,points.index('e')+1,1):
        if(points[i]=='e'):
            c=points.index('e')+1
            points[points.index('e')]='a'
            rotpoints.clear()
            counter=0
            break;
        else:
            p0=rotpoints[counter][0]
            p1=rotpoints[counter][1]
            i1=rotpoints[counter][2]
            pi=h+(p0-h)*round(math.cos(math.radians(-rotate[a])))-(p1-k)*round(math.sin(math.radians(-rotate[a])))
            pj=k+(p0-h)*round(math.sin(math.radians(-rotate[a])))-(p1-k)*round(math.cos(math.radians(-rotate[a])))
            value[xy.index(str(pi)+","+str(pj))]=i1
            counter+=1

In [13]:
decipher=value[0:P]
Bytes = np.packbits(decipher)
str1=[]
for i in range(0,len(Bytes),1):
    try:
        str1.append(binascii.unhexlify("%x"%Bytes[i]))
    except: 
        str1.append(binascii.unhexlify("0%x" %Bytes[i]))
#print(str1)
text=b''.join(str1)
#print(text)

#print(text)

In [14]:
filename="dec."+extension
with open(filename,'wb') as f:
    f.write(text)
print("Successfully Decrypted..")